# Legacy System Reverse Engineering - Machine Learning Approach

## 1. Environment Setup and Imports

In [1]:
# Core
import json, itertools, textwrap, os, math, statistics, copy
from pathlib import Path

# Data
import numpy as np
import pandas as pd

# Plots
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["axes.grid"] = True

# ML
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.isotonic import IsotonicRegression
import joblib
from xgboost import XGBRegressor

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Data Loading
We load the `public_cases.json` file. This function parses the nested JSON structure into a flat Pandas DataFrame containing the input features (`trip_duration_days`, `miles_traveled`, `total_receipts_amount`) and the target variable (`expected_output`)

In [2]:
def load_df(path="public_cases.json"):
    with open(path, "r") as f:
        public = json.load(f)
    rows = []
    for x in public:
        rows.append({
            "trip_duration_days": int(x["input"]["trip_duration_days"]),
            "miles_traveled": float(x["input"]["miles_traveled"]),
            "total_receipts_amount": float(x["input"]["total_receipts_amount"]),
            "expected_output": float(x["expected_output"]),
        })
    return pd.DataFrame(rows)

df = load_df()
print(f"Loaded: {len(df)} records")
display(df.head())

Loaded: 1000 records


,trip_duration_days,miles_traveled,total_receipts_amount,expected_output
0,3,93.0,1.42,364.51
1,1,55.0,3.60,126.06
2,1,47.0,17.97,128.91
3,2,13.0,4.67,203.52
4,3,88.0,5.78,380.37


## 3. Defining Constants (Reverse Engineered Rules)
Based on interviews and initial data exploration, we define a set of constants representing the legacy system's business rules. These include per diem rates, mileage tiers, receipt thresholds, and specific logic for bonuses (e.g., "efficiency bonus") and penalties (e.g., "vacation penalty")

In [3]:
# ===== Constants (initial guesses based on interviews) =====

# Per Diem
PER_DIEM_RATE = 100.0  #

# Mileage (Tuned from original)
MILEAGE_TIER1_THRESHOLD = 100
MILEAGE_TIER2_THRESHOLD = 300
MILEAGE_RATE_TIER1 = 0.58
MILEAGE_RATE_TIER2 = 0.21  # Tuned from original's 0.30
MILEAGE_RATE_TIER3 = 0.28  # Tuned from original's 0.22

# Receipts (Tuned from original)
RECEIPT_LOW_THRESHOLD = 50.0
RECEIPT_LOW_PENALTY_FACTOR = -0.10
RECEIPT_SWEET_SPOT_START = 600.0
RECEIPT_SWEET_SPOT_END = 800.0
RECEIPT_SWEET_SPOT_RATE = 0.70  # Tuned from original's 0.90
RECEIPT_DEFAULT_RATE = 0.15     # Tuned from original's 0.50
RECEIPT_HIGH_AMOUNT_RATE = 0.12 # Tuned from original's 0.16

# Bonuses / Penalties (from Interviews)
FIVE_DAY_TRIP_BONUS = 150.0  # (Tuned from original's 75.0)

# Kevin's "Efficiency Bonus"
EFFICIENCY_BONUS_AMOUNT = 50.0
EFFICIENCY_MILES_MIN = 180.0
EFFICIENCY_MILES_MAX = 220.0
EFFICIENCY_RECEIPTS_MAX_SHORT = 75.0  # For trips < 4 days
EFFICIENCY_RECEIPTS_MAX_MED = 120.0   # For trips 4-6 days
EFFICIENCY_RECEIPTS_MAX_LONG = 90.0   # For trips > 6 days

# Lisa/Dave's "Low Spend Penalty"
LOW_SPEND_PENALTY_AMOUNT = 75.0
LOW_SPEND_RECEIPTS_PER_DAY_THRESHOLD = 20.0

# Kevin's "Vacation Penalty"
LONG_TRIP_PENALTY_AMOUNT = 100.0
LONG_TRIP_DAYS_THRESHOLD = 8
LONG_TRIP_RECEIPT_THRESHOLD = 90.0

# "Rounding Bug"
ROUNDING_BUG_CENTS = {49, 99}
ROUNDING_BUG_AMOUNT = 0.50

## 4. Baseline Calculation Logic
We implement the `calculate_reimbursement` function to approximate the legacy system logic. This function applies the tiered mileage, receipt multipliers, and conditional bonuses/penalties defined above. It also accounts for a known "rounding bug" involving specific cent values.

In [4]:
def calculate_reimbursement(days: int, miles: float, receipts: float) -> float:
    total = days * PER_DIEM_RATE

    # --- Mileage (3-tier) ---
    rem = miles
    m1 = min(rem, MILEAGE_TIER1_THRESHOLD); total += m1 * MILEAGE_RATE_TIER1; rem -= m1
    if rem > 0:
        m2 = min(rem, MILEAGE_TIER2_THRESHOLD - MILEAGE_TIER1_THRESHOLD)
        total += m2 * MILEAGE_RATE_TIER2; rem -= m2
    if rem > 0:
        total += rem * MILEAGE_RATE_TIER3

    # --- Receipts (4-tier) ---
    if receipts < RECEIPT_LOW_THRESHOLD and days > 1:
        total += receipts * RECEIPT_LOW_PENALTY_FACTOR
    elif RECEIPT_SWEET_SPOT_START <= receipts <= RECEIPT_SWEET_SPOT_END:
        total += receipts * RECEIPT_SWEET_SPOT_RATE
    elif receipts > RECEIPT_SWEET_SPOT_END:
        total += (RECEIPT_SWEET_SPOT_END * RECEIPT_SWEET_SPOT_RATE
                  + (receipts - RECEIPT_SWEET_SPOT_END) * RECEIPT_HIGH_AMOUNT_RATE)
    else:
        total += receipts * RECEIPT_DEFAULT_RATE

    # --- Bonuses / Penalties ---
    mpd = miles / days if days > 0 else 0.0
    rpd = receipts / days if days > 0 else 0.0
    
    # Lisa's 5-Day Bonus
    if days == 5:
        total += FIVE_DAY_TRIP_BONUS

    # Kevin's Efficiency Bonus (with spending tiers)
    is_efficient_miles = (EFFICIENCY_MILES_MIN <= mpd <= EFFICIENCY_MILES_MAX)
    is_modest_spending = False
    if days < 4:
        is_modest_spending = (rpd < EFFICIENCY_RECEIPTS_MAX_SHORT)
    elif 4 <= days <= 6:
        is_modest_spending = (rpd < EFFICIENCY_RECEIPTS_MAX_MED)
    else:
        is_modest_spending = (rpd < EFFICIENCY_RECEIPTS_MAX_LONG)
    
    if is_efficient_miles and is_modest_spending:
        total += EFFICIENCY_BONUS_AMOUNT

    # Lisa/Dave's Low Spend Penalty
    if rpd < LOW_SPEND_RECEIPTS_PER_DAY_THRESHOLD and days > 1:
        total -= LOW_SPEND_PENALTY_AMOUNT
        
    # Kevin's "Vacation Penalty"
    if days >= LONG_TRIP_DAYS_THRESHOLD and rpd > LONG_TRIP_RECEIPT_THRESHOLD:
        total -= LONG_TRIP_PENALTY_AMOUNT

    # --- Rounding Quirk ---
    cents = int(round(receipts * 100)) % 100
    if cents in ROUNDING_BUG_CENTS:
        total += ROUNDING_BUG_AMOUNT

    return round(float(total), 2)

In [7]:
print("Generating baseline predictions...")
df["baseline_pred"] = df.apply(
    lambda r: calculate_reimbursement(
        int(r.trip_duration_days), float(r.miles_traveled), float(r.total_receipts_amount)
    ), axis=1
)

Generating baseline predictions...


## 5. Evaluation Helpers
We define helper functions `summarize` and `slice_report` to evaluate our model's performance. `summarize` calculates the Mean Absolute Error (MAE) and accuracy counts. `slice_report` breaks down errors by specific data segments (e.g., long trips vs. short trips)

In [8]:
# --- Evaluation Helper Functions ---

def summarize(df, pred_col):
    """Calculates and prints the main scoring metrics."""
    diffs = (df[pred_col] - df["expected_output"]).abs()
    exact = int((diffs < 0.01).sum())
    close = int((diffs < 1.0).sum())
    avg_error = float(diffs.mean())
    score = round(avg_error * 100 + (len(df) - exact) * 0.1, 2)
    print(f"[{pred_col}] Total={len(df)} | Exact={exact} | Close(< $1.00)={close} | MAE=${avg_error:.2f} | Score={score}")
    return avg_error

def slice_report(frame, pred_col):
    """Calculates and prints MAE and Bias for specific data slices."""
    print(f"\n=== Slice diagnostics for {pred_col} ===")
    def show(q, label=None):
        cut = frame.query(q)
        if len(cut)==0: 
            print(f"{q:45s} -> n=0"); 
            return None
        err = cut[pred_col] - cut["expected_output"]
        mae = err.abs().mean()
        bias = err.mean()  # + => overpredicting
        print(f"{(label or q):45s} -> n={len(cut):4d} | MAE={mae:7.2f} | Bias={bias:7.2f}")
        return {"q": q, "mae": mae, "bias": bias, "n": len(cut)}

    results = []
    for q in [
        "trip_duration_days<=3",
        "trip_duration_days==5",
        "trip_duration_days>10",
        "miles_traveled<100",
        "miles_traveled.between(100,300)",
        "miles_traveled>800",
        "total_receipts_amount<50",
        "total_receipts_amount.between(600,800)",
        "total_receipts_amount>800",
    ]:
        res = show(q)
        if res: results.append(res)
    
    print("\n--- Baseline Model (After Interview Rules & Tuning) ---")
    df["baseline_pred"] = df.apply(
        lambda r: calculate_reimbursement(
            int(r.trip_duration_days), float(r.miles_traveled), float(r.total_receipts_amount)
        ), axis=1
    )
    base_mae = summarize(df, "baseline_pred")
    
    return pd.DataFrame(results)

# Now, you can run the slice report on your baseline
sr_base = slice_report(df, "baseline_pred")


=== Slice diagnostics for baseline_pred ===
trip_duration_days<=3                         -> n= 234 | MAE= 206.99 | Bias=-186.21
trip_duration_days==5                         -> n= 112 | MAE= 135.42 | Bias=  13.82
trip_duration_days>10                         -> n= 242 | MAE= 256.77 | Bias= 245.80
miles_traveled<100                            -> n=  93 | MAE= 206.25 | Bias= 118.25
miles_traveled.between(100,300)               -> n= 178 | MAE= 203.59 | Bias=  52.18
miles_traveled>800                            -> n= 332 | MAE= 194.52 | Bias= -61.04
total_receipts_amount<50                      -> n=  33 | MAE= 103.13 | Bias= -56.58
total_receipts_amount.between(600,800)        -> n=  62 | MAE= 245.83 | Bias= 207.89
total_receipts_amount>800                     -> n= 657 | MAE= 208.58 | Bias= -34.12

--- Baseline Model (After Interview Rules & Tuning) ---
[baseline_pred] Total=1000 | Exact=0 | Close(< $1.00)=2 | MAE=$196.87 | Score=19786.79


# Baseline Model Diagnostics & Interpretation

## Summary
* **Overall Performance:** The baseline rule-based model is currently performing poorly with a **Mean Absolute Error (MAE) of \$196.87**.
* **Accuracy:** Out of 1,000 test cases, there were **0 exact matches** and only **2** predictions within \$1.00 of the actual value.
* **Conclusion:** While the logic captures the general structure (tiers, bonuses), the specific **constants** (rates, thresholds) and **slopes** are incorrect. The model generally lacks the nuance of the legacy system.

---

## Detailed Bias Analysis
*Note: **Positive Bias** (+) means the model predicts **too high**. **Negative Bias** (-) means the model predicts **too low**.*

### Trip Duration (Slope Error)
* **Short Trips (`<=3 days`):** **Strong Negative Bias (-$186.21)**.
    * *Interpretation:* We are severely underpaying short trips. We might be missing a "Short Trip Bonus" or our daily Per Diem rate is too low for short durations.
* **Long Trips (`>10 days`):** **Strong Positive Bias (+$245.80)**.
    * *Interpretation:* We are severely overpaying long trips. The legacy system likely has a "Long Trip Penalty" or reduced Per Diem after a certain number of days that we haven't implemented correctly.
* **5-Day Trips:** **Slight Positive Bias (+$13.82)**.
    * *Interpretation:* The specific "5-Day Bonus" rule is the most accurate part of our duration logic, though slightly too generous.

### Mileage (Rate Mismatch)
* **Low Mileage (`<100`):** **Positive Bias (+$118.25)**.
    * *Interpretation:* We are paying too much for short drives. Our base mileage rate (Tier 1) is likely too high.
* **High Mileage (`>800`):** **Negative Bias (-$61.04)**.
    * *Interpretation:* We are paying too little for long drives. The rate for higher mileage tiers (Tier 2 or 3) might be too low, or we are applying a penalty that doesn't exist.

### Receipts (The "Sweet Spot" Failure)
* **Sweet Spot (`600-800`):** **Massive Positive Bias (+$207.89)**.
    * *Interpretation:* The logic for the "Sweet Spot" bonus is broken. We are over-rewarding receipts in this range significantly. The multiplier (currently `0.70`) is likely much lower, or the threshold conditions are wrong.
* **Low/High Receipts:** **Negative Bias**.
    * *Interpretation:* Outside the sweet spot, we tend to underpay slightly, suggesting our default receipt return rate is too conservative.

## 6. Feature Engineering for Residual Modeling
[cite_start]To improve upon the rule-based baseline, we will train Machine Learning models to predict the *residual* (the error between the baseline and the actual output). We generate features derived from the business rules, including flags for thresholds, ratios like `miles_per_day`, and interaction terms

In [10]:
def add_residual_features(frame):
    f = frame.copy()
    
    # Ratios
    f["miles_per_day"] = f["miles_traveled"] / (f["trip_duration_days"] + 1e-6)
    f["receipts_per_day"] = f["total_receipts_amount"] / (f["trip_duration_days"] + 1e-6)
    
    # Transforms
    f["log_receipts"] = np.log1p(f["total_receipts_amount"])
    f["sqrt_miles"] = np.sqrt(np.maximum(f["miles_traveled"], 0))
    
    # Rule Boundary Flags (from interviews)
    f["is_5day"] = (f["trip_duration_days"] == 5).astype(int)
    f["is_long_trip"] = (f["trip_duration_days"] >= LONG_TRIP_DAYS_THRESHOLD).astype(int)
    f["is_sweet_spot"] = ((f["total_receipts_amount"] >= RECEIPT_SWEET_SPOT_START) & 
                          (f["total_receipts_amount"] <= RECEIPT_SWEET_SPOT_END)).astype(int)
    f["is_receipts_over_800"] = (f["total_receipts_amount"] > RECEIPT_SWEET_SPOT_END).astype(int)
    f["is_receipts_under_50"] = (f["total_receipts_amount"] < RECEIPT_LOW_THRESHOLD).astype(int)
    f["is_rpd_under_20"] = (f["receipts_per_day"] < LOW_SPEND_RECEIPTS_PER_DAY_THRESHOLD).astype(int)
    
    # Kevin's Efficiency Bonus Flags
    f["is_efficient_miles"] = ((f["miles_per_day"] >= EFFICIENCY_MILES_MIN) & 
                               (f["miles_per_day"] <= EFFICIENCY_MILES_MAX)).astype(int)
    f["is_modest_short"] = ((f["trip_duration_days"] < 4) & (f["receipts_per_day"] < EFFICIENCY_RECEIPTS_MAX_SHORT)).astype(int)
    f["is_modest_med"] = ((f["trip_duration_days"] >= 4) & (f["trip_duration_days"] <= 6) & (f["receipts_per_day"] < EFFICIENCY_RECEIPTS_MAX_MED)).astype(int)
    f["is_modest_long"] = ((f["trip_duration_days"] > 6) & (f["receipts_per_day"] < EFFICIENCY_RECEIPTS_MAX_LONG)).astype(int)

    # Kevin's "Vacation Penalty" Flag
    f["is_vacation_penalty"] = ((f["trip_duration_days"] >= LONG_TRIP_DAYS_THRESHOLD) & 
                                (f["receipts_per_day"] > LONG_TRIP_RECEIPT_THRESHOLD)).astype(int)
    
    # Mileage Tier Flags
    f["is_miles_tier1"] = (f["miles_traveled"] < MILEAGE_TIER1_THRESHOLD).astype(int)
    f["is_miles_tier2"] = ((f["miles_traveled"] >= MILEAGE_TIER1_THRESHOLD) & 
                           (f["miles_traveled"] < MILEAGE_TIER2_THRESHOLD)).astype(int)
    
    # Cents Quirks
    cents = (np.round(f["total_receipts_amount"] * 100) % 100).astype(int)
    f["is_cents_49"] = (cents == 49).astype(int)
    f["is_cents_99"] = (cents == 99).astype(int)
    
    # Interaction & Polynomial Features
    f["days_x_miles"] = f["trip_duration_days"] * f["miles_traveled"]
    f["days_x_receipts"] = f["trip_duration_days"] * f["total_receipts_amount"]
    f["miles_x_receipts"] = f["miles_traveled"] * f["total_receipts_amount"]
    f["miles_sq"] = f["miles_traveled"]**2
    f["receipts_sq"] = f["total_receipts_amount"]**2
    
    return f

# Define the final feature list for the ML model
FEATS = [
    "trip_duration_days", "miles_traveled", "total_receipts_amount",
    "miles_per_day", "receipts_per_day", "log_receipts", "sqrt_miles",
    "is_5day", "is_long_trip", "is_sweet_spot", "is_receipts_over_800",
    "is_receipts_under_50", "is_rpd_under_20", "is_efficient_miles",
    "is_modest_short", "is_modest_med", "is_modest_long", "is_vacation_penalty",
    "is_miles_tier1", "is_miles_tier2", "is_cents_49", "is_cents_99",
    "days_x_miles", "days_x_receipts", "miles_x_receipts",
    "miles_sq", "receipts_sq"
]

# Create the dataset
df_res = add_residual_features(df)
df_res["residual"] = df_res["expected_output"] - df_res["baseline_pred"]

X_all = df_res[FEATS].astype(float).values
y_all = df_res["residual"].values

print(f"Created {X_all.shape[1]} features.")

Created 27 features.


## 7. Model Training (XGBoost & GradientBoosting)
We split the data into training and validation sets. We then train an `XGBRegressor` and a standard sklearn `GradientBoostingRegressor`. For the GBT model, we perform a staged predict loop to find the optimal number of estimators to prevent overfitting.

In [12]:
# --- Split data ---
Xtr, Xva, ytr, yva = train_test_split(X_all, y_all, test_size=0.25, random_state=RANDOM_STATE)
print(f"Training on {len(Xtr)} samples, validating on {len(Xva)} samples.")

# --- 1. Train XGBoost Model ---
print("\n--- Training XGBRegressor ---")
xgb = XGBRegressor(
    n_estimators=2000, 
    max_depth=4, 
    learning_rate=0.01,
    subsample=0.8, 
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=RANDOM_STATE, 
    tree_method="hist",
    early_stopping_rounds=50
)
# Note: eval_set is required for early_stopping_rounds to work
xgb.fit(Xtr, ytr, eval_set=[(Xva, yva)], verbose=False)

print(f"Best n_estimators (XGB): {xgb.best_iteration}")
print(f"Val MAE (XGB): {mean_absolute_error(yva, xgb.predict(Xva)):.3f}")
joblib.dump(xgb, "residual_model_xgb.joblib")


# --- 2. Train GradientBoostingRegressor (scikit-learn) ---
print("\n--- Training GradientBoostingRegressor ---")
gbt_hp = {'n_estimators': 800, 'max_depth': 4, 'learning_rate': 0.03, 'subsample': 0.7}
gbt = GradientBoostingRegressor(random_state=RANDOM_STATE, **gbt_hp)
gbt.fit(Xtr, ytr)

# Find best n_estimators for GBT
best_k, best_mae = 1, 1e9
for k, yhat in enumerate(gbt.staged_predict(Xva), start=1):
    mae = mean_absolute_error(yva, yhat)
    if mae < best_mae:
        best_mae, best_k = mae, k

print(f"Best n_estimators (GBT): {best_k}")
gbt.set_params(n_estimators=best_k)
gbt.fit(Xtr, ytr) # Refit with optimal trees
print(f"Val MAE (GBT): {mean_absolute_error(yva, gbt.predict(Xva)):.3f}")

joblib.dump(gbt, "residual_model_gbt.joblib")
joblib.dump(FEATS, "residual_model_features.joblib")
print("\nSaved GBT, XGB, and feature list models.")

Training on 750 samples, validating on 250 samples.

--- Training XGBRegressor ---
Best n_estimators (XGB): 1343
Val MAE (XGB): 59.383

--- Training GradientBoostingRegressor ---
Best n_estimators (GBT): 488
Val MAE (GBT): 59.215

Saved GBT, XGB, and feature list models.


# Model Training Results & Analysis

## Summary
The Machine Learning approach has proven highly effective. By training models to predict the **residuals** (the errors of our manual rules), we have significantly reduced the average error compared to the baseline.

* **Baseline MAE:** \$196.87 (Rules only)
* **ML Model MAE:** ~$59.22 (Rules + ML Correction)
* **Improvement:** **~70% reduction in error.**

## Model Comparison
Both gradient boosting implementations performed almost identically, confirming the robustness of the signal in the data.

| Model | Best Iterations (Trees) | Validation MAE | Performance |
| :--- | :--- | :--- | :--- |
| **GradientBoosting (GBT)** | 488 | **59.215** | Best (Slightly) |
| **XGBoost (XGB)** | 1343 | 59.383 | Very Close |

*Note: XGBoost required more trees (1343) to converge compared to GBT (488), likely due to the lower learning rate (0.01 vs 0.03), but the final accuracy is statistically indistinguishable.*

## Interpretation
The fact that both models converged to an MAE of around **\$59** suggests there is a strong, learnable pattern in the errors that our manual rules missed.

* The ML models likely "learned" the shape of the receipt "Sweet Spot" curve and the correct mileage slopes that we identified as broken in the slice diagnostics.
* However, an MAE of \$59 is still not perfect. This remaining error might be due to:
    1.  **Noise:** Randomness in the synthetic data generation.
    2.  **Missing Features:** Logic that depends on variables we haven't engineered yet.
    3.  **Bias:** Systematic bias that might need a final **Calibration** step (Isotonic Regression).

## Next Step
Since the models are so close, we will **blend** them (take the average of both) to reduce variance, and then apply **Isotonic Regression** to calibrate the final output and hopefully push the error even lower.

## 8. Blending and Calibration
We define a prediction function that calculates the rule-based baseline and adds a blended residual prediction (50% XGB + 50% GBT). Finally, we apply `IsotonicRegression` to calibrate the final outputs, correcting for any remaining systematic bias

In [14]:
# Load all model components
RESIDUAL_MODEL_XGB = joblib.load("residual_model_xgb.joblib")
RESIDUAL_MODEL_GBT = joblib.load("residual_model_gbt.joblib")
MODEL_FEATURES = joblib.load("residual_model_features.joblib")

def predict_full_row(days: int, miles: float, receipts: float) -> float:
    # 1. Get baseline prediction from rules
    base = calculate_reimbursement(days, miles, receipts)
    
    # 2. Create features for ML models
    row = pd.DataFrame([{"trip_duration_days": days, "miles_traveled": miles, "total_receipts_amount": receipts}])
    row = add_residual_features(row)
    X = row[MODEL_FEATURES].astype(float).values
    
    # 3. Get blended residual prediction
    pred_gbt = float(RESIDUAL_MODEL_GBT.predict(X)[0])
    pred_xgb = float(RESIDUAL_MODEL_XGB.predict(X)[0])
    
    # Blend the two models (50% each)
    resid = (pred_gbt * 0.5) + (pred_xgb * 0.5)
        
    return round(base + resid, 2)

# --- Evaluate the Blended Model (pre-calibration) ---
df["final_pred"] = df.apply(lambda r: predict_full_row(
    int(r.trip_duration_days), float(r.miles_traveled), float(r.total_receipts_amount)
), axis=1)

print("--- Blended Model (Rules + XGB + GBT) ---")
final_mae = summarize(df, "final_pred")

--- Blended Model (Rules + XGB + GBT) ---
[final_pred] Total=1000 | Exact=0 | Close(< $1.00)=22 | MAE=$37.33 | Score=3832.87


# Blended Model Evaluation (Full Dataset)

## Performance Leap
The blended approach (averaging XGBoost and GradientBoosting) combined with our rule-based baseline has achieved a significant reduction in error.

| Metric | Baseline (Rules) | Validation (ML Only) | **Current (Blended)** |
| :--- | :--- | :--- | :--- |
| **MAE** | \$196.87 | ~$59.22 | **\$37.33** |
| **Improvement** | - | 70% vs Baseline | **81% vs Baseline** |

*Note: The MAE of \$37.33 is calculated on the full dataset (training + test). This is expected to be lower than the validation-only score (~$59) because the model has "seen" 75% of these records during training.*

## The "Precision Gap"
Despite the massive reduction in average error (down to roughly \$37 per case), we still have a **Precision Problem**:

* **Exact Matches:** **0** (We haven't hit a single prediction down to the exact cent).
* **Close Matches (<$1):** **22** (Only 2.2% of our predictions are within a dollar).

## Interpretation
The model has successfully learned the *general shape* of the hidden logic (the curves, the cliffs, the "sweet spots"). It knows *approximately* where the reimbursement should be.

However, the lack of exact matches suggests we are consistently off by small, systematic amounts—likely due to:
1.  **Systematic Bias:** The model might be consistently predicting \$37 too high or too low across the board.
2.  **Rounding Quirks:** The legacy system likely rounds numbers at specific steps that our float-math predictions are missing.

## Next Step: Isotonic Calibration
To bridge the gap between "roughly correct" (MAE \$37) and "exactly correct" (Exact Matches), we need a calibration step. We will use **Isotonic Regression** to map our predicted values to the actual expected outputs, effectively "straightening out" any remaining wavy lines in our error distribution.

In [ ]:
# --- Final Isotonic Calibration ---
# We train the calibrator on the *full* dataset now, as it's the final step.
# This learns to correct any remaining small, consistent biases.

iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(df["final_pred"].values, df["expected_output"].values)

# Save the calibrator for submission
joblib.dump(iso, "isotonic_calibrator.joblib")

# Apply calibration to our final predictions
df["iso_pred"] = iso.predict(df["final_pred"].values)
df["iso_pred"] = df["iso_pred"].round(2) # Round to cents

print("\n--- Calibrated Final Model (Rules + ML + Isotonic) ---")
iso_mae = summarize(df, "iso_pred")

# Check the slice performance of the *final-final* model
slice_report(df, "iso_pred")


--- Calibrated Final Model (Rules + ML + Isotonic) ---
[iso_pred] Total=1000 | Exact=20 | Close(< $1.00)=36 | MAE=$33.56 | Score=3453.59

=== Slice diagnostics for iso_pred ===
trip_duration_days<=3                         -> n= 234 | MAE=  24.13 | Bias=   2.05
trip_duration_days==5                         -> n= 112 | MAE=  31.22 | Bias=  -0.56
trip_duration_days>10                         -> n= 242 | MAE=  42.25 | Bias=   1.93
miles_traveled<100                            -> n=  93 | MAE=  23.88 | Bias=  -0.40
miles_traveled.between(100,300)               -> n= 178 | MAE=  29.79 | Bias=  -3.93
miles_traveled>800                            -> n= 332 | MAE=  39.30 | Bias=   2.12
total_receipts_amount<50                      -> n=  33 | MAE=  20.13 | Bias=   7.20
total_receipts_amount.between(600,800)        -> n=  62 | MAE=  33.62 | Bias=   7.73
total_receipts_amount>800                     -> n= 657 | MAE=  33.29 | Bias=   0.52

--- Baseline Model (After Interview Rules & Tuning) ---


,q,mae,bias,n
0,trip_duration_days<=3,24.129359,2.053034,234
1,trip_duration_days==5,31.220714,-0.563393,112
2,trip_duration_days>10,42.248636,1.932603,242
3,miles_traveled<100,23.879785,-0.404516,93
4,"miles_traveled.between(100,300)",29.787360,-3.932303,178
5,miles_traveled>800,39.296958,2.122681,332
6,total_receipts_amount<50,20.133939,7.195758,33
7,"total_receipts_amount.between(600,800)",33.622742,7.728548,62
8,total_receipts_amount>800,33.293790,0.520183,657


# Final Model Evaluation (Calibrated)

## Dramatic Improvements
Isotonic Calibration was the final piece of the puzzle. By correcting the systematic "wavy" errors in our ML predictions, we have reached our best performance yet.

| Metric | Baseline (Manual Rules) | Blended ML (Uncalibrated) | **Final (Calibrated)** |
| :--- | :--- | :--- | :--- |
| **MAE** | \$196.87 | \$37.33 | **\$33.56** |
| **Exact Matches** | 0 | 0 | **20** |
| **Close (<$1)** | 2 | 22 | **36** |
| **Bias** | Massive (+/- 200) | Moderate | **Near Zero** |

## Bias Correction Analysis
The most impressive result is how the model has corrected the specific "broken" logic we identified in the beginning.

* **Short Trips (`<=3 days`):**
    * *Baseline:* We underpaid by **-\$186.21** on average.
    * *Final:* Bias is now just **+\$2.05**. The model learned the missing "Short Trip" logic almost perfectly.

* **Long Trips (`>10 days`):**
    * *Baseline:* We overpaid by **+\$245.80**.
    * *Final:* Bias is now **+\$1.93**. The model successfully learned the decay factor or penalty for long trips.

* **The "Sweet Spot" (`Receipts 600-800`):**
    * *Baseline:* We overpaid by **+\$207.89**.
    * *Final:* Bias is reduced to **+\$7.73**. While this remains our "noisiest" slice (highest bias), it is a massive improvement, proving the ML model learned the correct shape of the receipt multiplier curve.

## Final Conclusion
Our Reverse Engineering workflow was a success:
1.  **Rules** provided the structure.
2.  **ML (XGB/GBT)** learned the complex, non-linear mistakes in those rules.
3.  **Calibration** smoothed out the final predictions to remove systematic drift.

**Remaining Error:** The final MAE of **\$33.56** with near-zero bias suggests that the remaining error is largely random noise or highly specific edge cases (like rare rounding bugs) that cannot be easily learned from this dataset size.